In [16]:
#########################
###Lotta Revised Score###
#########################

#Description: To use Scott beta coefficients when available; otherwise use Manning
#Revision: 

#install packages
library(utils)
library(psych)
library(rms)
library(naniar)
library(tidyverse)
library(table1)
library(readr)
library(stringr)
library(R.utils)
library(data.table)
library(dplyr)
#library(sqldf)

#load beta coefficients
#Scott if it's available, otherwise Manning (larger N in Scott, Manning: smaller N but full GWAS)
SCOTT_m <- fread("~/jupyter/IRKD_SNP/MAGIC_beta/MAGIC_Scott_et_al_FI_adjBMI_Jan2013.txt", header = TRUE, na.strings=c("",".","NA"))
SCOTT=as.data.frame(SCOTT_m)
rm(SCOTT_m)
MAN_m <- fread("~/jupyter/IRKD_SNP/MAGIC_beta/MAGIC_Manning_et_al_lnFastingInsulin_MainEffect.txt", header = TRUE, na.strings=c("",".","NA"))
MAN=as.data.frame(MAN_m)
rm(MAN_m)
#lotta coefficients (18-SNP version)
LOTTA <- read.table(paste0("/data/nrnb03/users/agarduno/jupyter/IRKD_SNP/IR_LOTTA_Oct2420_v3.txt"), header=TRUE)
names(LOTTA)=c("rsid", "EA", "NEA", "Beta")
#lotta coefficients (53-SNP version)
LOTTA53 <- read.table(paste0("/data/nrnb03/users/agarduno/jupyter/IRKD_SNP/IR_LOTTA_05Mar21.csv"), header=TRUE,sep = ',')
names(LOTTA53)=c("rsid", "EA", "NEA", "Beta")

#function: flip the effect and ref allele when flipping beta
nobeta_flip <- function(coef_SCLOT) {
#flip the effect allele to the reference allele (this is setting the effect allele when beta is negative)
coef_SCLOT$EA2 <- coef_SCLOT$effect_allele))
coef_SCLOT$NEA2 <- ifelse(coef_SCLOT$beta < 0,as.character(coef_SCLOT$effect_allele),as.character(coef_SCLOT$other_allele))
#convert to upper case
coef_SCLOT$EA2 <- toupper(coef_SCLOT$EA2)
coef_SCLOT$NEA2 <- toupper(coef_SCLOT$NEA2)
coef_SCLOT$beta <- ifelse(coef_SCLOT$beta < 0,coef_SCLOT$beta*-1,coef_SCLOT$beta) #flip beta
drop <- c("Beta","effect_allele","other_allele","EA","NEA")
print(names(coef_SCLOT))
coef_SCLOT  = coef_SCLOT[,!(names(coef_SCLOT) %in% drop)]    
coef_SCLOT <- rename(coef_SCLOT,EA=EA2)
coef_SCLOT <- rename(coef_SCLOT,NEA=NEA2)
print("flip")
return(coef_SCLOT)
}

#function: replace lotta beta coefficients with scott and manning
lotta_scott <- function(LOTTA) {
    #only allow Lotta to be rsid
    rsid <- LOTTA$rsid
    LOTTA2 <- as.data.frame(rsid)
    #coefficients (Scott & Lotta), limited to the primary 18 SNPs
    SCOTT <- rename(SCOTT,rsid=snp)
    MAN <- rename(MAN,rsid=Snp)
    coef_SCLOT <- merge(SCOTT,LOTTA2,by="rsid",suffixes = c(".scott",".lotta")) #17/18 snps
    #flip coefficient for Scott
    coef_SCLOT2 <- beta_flip(coef_SCLOT)

    #coefficients (Scott & Manning), limited to the primary 17 SNPs
    coef_MNLOT <- merge(MAN,LOTTA2,by="rsid",suffixes = c(".man",".lotta")) #17/18 snps
    #generate new dataframe with coefficients of interest
    drop <- c("MainEffects","MainSE","MainSE","MainP")
    coef_MNLOTT  = coef_MNLOT [,!(names(coef_MNLOT) %in% drop)]
    coef_MNLOTT = rename(coef_MNLOTT,beta=BMIadjMainEffects)
    coef_MNLOTT = rename(coef_MNLOTT,se=BMIadjMainSE)
    #flip for effect increasing
    coef_MNLOT2 <- beta_flip(coef_MNLOTT)

    #Generate summary SNP list
    #sort
    coef_SCLOT2 <- coef_SCLOT2[order(coef_SCLOT2$rsid),]
    coef_MNLOT2 <- coef_MNLOT2[order(coef_MNLOT2$rsid),]
    coef_MNLOT2 <- coef_MNLOT2[c(1,3,4,5,2,6,7)]
    coef_MNLOT2 <- rename(coef_MNLOT2,pvalue=BMIadjMainP)

    #pull only snp not in scott
    coef_MNLOT3 <- coef_MNLOT2[!(coef_MNLOT2$rsid %in% coef_SCLOT2$rsid),]
    print(coef_MNLOT3)
    #final snp_list
    final_list <- rbind(coef_SCLOT2,coef_MNLOT3)
    return(final_list)
}

#export scott and manning list (18-SNP version)
final_list <- lotta_scott(LOTTA)
#write csv
write.csv(final_list,'MAGIC_LottaSnps_ScottMann_23Aug2021.csv')
#export scott and manning list (53-SNP version)
final_list53 <- lotta_scott(LOTTA53)
write.csv(final_list53,'MAGIC_LottaSnps53_ScottMann_23Aug2021.csv')

[1] "rsid"          "effect_allele" "other_allele"  "beta"         
[5] "se"            "pvalue"        "maf"           "EA2"          
[9] "NEA2"         
[1] "flip"
[1] "rsid"          "effect_allele" "other_allele"  "maf"          
[5] "beta"          "se"            "BMIadjMainP"   "EA2"          
[9] "NEA2"         
[1] "flip"
       rsid  beta     se    pvalue  maf EA NEA
18 rs972283 0.013 0.0027 4.408e-06 0.45  G   A
[1] "rsid"          "effect_allele" "other_allele"  "beta"         
[5] "se"            "pvalue"        "maf"           "EA2"          
[9] "NEA2"         
[1] "flip"
[1] "rsid"          "effect_allele" "other_allele"  "maf"          
[5] "beta"          "se"            "BMIadjMainP"   "EA2"          
[9] "NEA2"         
[1] "flip"
         rsid   beta     se    pvalue   maf EA NEA
5  rs11130329 0.0140 0.0040 5.117e-04 0.159  A   C
12 rs17402950 0.0160 0.0057 4.682e-03 0.042  G   A
42  rs7227237 0.0099 0.0031 1.331e-03 0.186  C   T
44  rs7323406 0.0090 0.0030 2.674e